# Dataset Construction

This is mostly a scratchpad area.  I use it to examine, and then subset the extended311 to generate smaller data sets.

I am going to start by reading all the data from the files generated in 311-data-cleaning and 311-data-cleaning-part2

Two of the sections are required for other notebooks. ecwandc section generates data for ECWANDC.ipynb and graffiti for graffiti.ipynb.

**Note:**  Documentation/narrative in the nb is a bit lacking!

In [ ]:
%run start.py
import nc

from utils import read_new311_shape, dt_to_object

Reading the various files generated in cleaning with quick examination to see if they make sense.

In [ ]:
%%time
clean311_df = pd.read_csv('../data/311/clean311.csv', low_memory=False)

In [ ]:
clean311_df.info()

(01/18/2022)  Changing this up to reflect paraquet files.

If you want the other formats then uncomment here and build appropriate in 311-data-cleaning.ipynb.

In [ ]:
#clean311_geojson_gdf = read_new311_shape('../data/311/clean311.geojson')

#clean311_geojson_gdf.info()

In [ ]:
#clean311_geo_shape_gdf = read_new311_shape('../data/311/clean311-geo.shp')

#clean311_geo_shape_gdf.info()

In [ ]:
#clean311_regions_geojson_gdf = read_new311_shape('../data/311/clean311-regions.geojson')
#clean311_regions_geojson_gdf.info()

In [ ]:
#clean311_regions_shape_gdf = read_new311_shape('../data/311/clean311-regions.shp')
#clean311_regions_shape_gdf.info()

So these tests seem reasonable.

This is the file used in other parts of the nb to examine, subset, ...

In [ ]:
%%time
clean311_gdf = gpd.read_parquet('../data/311/clean311.parq')

In [ ]:
clean311_gdf.info()

So this parquet format is quite amazing.  It saves the dtypes I want.  If is fast and efficient.

In [ ]:
#extended311_gdf = read_new311_shape('../data/311/extended311-geo-shape.shp')
#extended311_gdf.info()

In [ ]:
extended311_gdf = gpd.read_parquet('../data/311/extended311-geo-shape.parq')
extended311_gdf.info()

# Graffiti 

Some experimentation here.  This is the use case for the first workshop.

Use this as an exemplar if you want to try different request_types.

In [ ]:
extended311_gdf['request_type'].value_counts()

In [ ]:
graffiti_gdf = extended311_gdf.query(f"request_type == 'Graffiti Removal'").reset_index().drop(columns=['index'])

In [ ]:
graffiti_gdf['nc_name'].value_counts()

In [ ]:
graffiti_gdf['service_region'].value_counts()

In [ ]:
#graffiti_gdf = dt_to_object(graffiti_gdf)

#graffiti_gdf.to_file('../data/311/graffiti.geojson', driver='GeoJSON')

In [ ]:
graffiti_gdf.to_parquet('../data/311/graffiti.parq')

This is still a bit over 500MB.  Too big.

Holy crap.  The parquet file is 33M.

# Looking at Service Regions

I'm basically trying to find some that are under the 100MB limit for github.

**Note:** Since the new approach focuses on generating the 311 data sets this is probably moot.  

In [ ]:
extended311_gdf.service_region.value_counts()

number of 311 calls per region

In [ ]:
extended311_gdf.region_id.value_counts()

In [ ]:
region_gdf = extended311_gdf.query(f"region_id == 7", engine="python").reset_index().drop(columns=['index'])

In [ ]:
len(region_gdf)

In [ ]:
#region_gdf = dt_to_object(region_gdf)

In [ ]:
#region_gdf.to_file('../data/311/region.zip')

In [ ]:
region_gdf.to_parquet('../data/311/region.parq')

In [ ]:
region_gdf.request_type.value_counts()

In [ ]:
region_gdf.nc_name.value_counts()

In [ ]:
region2_gdf = extended311_gdf.query(f"region_id == 2", engine="python").reset_index().drop(columns=['index'])
region2_gdf = dt_to_object(region2_gdf)

In [ ]:
#region2_gdf.to_file('../data/311/region2.zip')
region2_gdf.to_parquet('../data/311/region2.parq')

In [ ]:
extended311_gdf.nc.value_counts()

In [ ]:
region78_gdf = extended311_gdf.query(f"nc == 78", engine="python").reset_index().drop(columns=['index'])

In [ ]:
region78_gdf.to_parquet('../data/311/region78.parq')

In [ ]:
extended311_gdf[['address']].value_counts()

In [ ]:
groupby_request_type = extended311_gdf.groupby('request_type')

In [ ]:
len(groupby_request_type)

In [ ]:
groupby_request_type.get_group('Bulky Items').reset_index().drop(columns=['index'])

# ecwandc

I thought this NC may be a good one to explore.  They had the most participants in the data101 presentations.

I use this file in the ECWANDC.ipynb analysis.

In [ ]:
#small_gdf = new311_gdf.query(f"nc == 46").reset_index().drop(columns=['index'])
ecwandc311_gdf = extended311_gdf.query(f"nc == 79", engine="python").reset_index().drop(columns=['index'])

In [ ]:
len(ecwandc311_gdf)

In [ ]:
ecwandc311_gdf.info()

Finally, for this data set I'm going to save as a basic shape file.  It's small so should fit within size limits.

In [ ]:
#ecwandc311_gdf = dt_to_object(ecwandc311_gdf)

#ecwandc311_gdf.to_file('../data/311/ecwandc311.shp')

In [ ]:
ecwandc311_gdf.to_parquet('../data/311/ecwandc311.parq')